In [11]:
!pip install duckdb ydata-profiling pycaret  xgboost lightgbm catboost

^C


In [178]:
import pandas as pd
import numpy as np
import duckdb
import plotnine as p9

import pandas as pd
from sklearn.preprocessing import KBinsDiscretizer
from kmodes.kmodes import KModes

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from kmodes.kprototypes import KPrototypes
from sklearn.metrics import adjusted_rand_score


In [179]:
full_train_csv_data = pd.read_csv("playground-series-s4e12/train.csv")
full_test_csv_data = pd.read_csv("playground-series-s4e12/test.csv")

In [180]:
len(full_test_csv_data)

800000

In [181]:
len(full_train_csv_data)

1200000

In [182]:
import pandas as pd
from pycaret.regression import setup, compare_models, add_metric

In [183]:
# Separate numeric and categorical columns
numeric_cols = full_test_csv_data.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = full_test_csv_data.select_dtypes(include=['object']).columns

In [184]:
categorical_cols

Index(['Gender', 'Marital Status', 'Education Level', 'Occupation', 'Location',
       'Policy Type', 'Policy Start Date', 'Customer Feedback',
       'Smoking Status', 'Exercise Frequency', 'Property Type'],
      dtype='object')

In [185]:
def treat_dataset_pandas_init(dataset):
    dataset = dataset.copy()
    dataset['Policy Start Date'] = pd.to_datetime(dataset['Policy Start Date'])
    # Get the last date in the dataset
    last_date = dataset['Policy Start Date'].max()
    
    # Add 1 day to the last date
    reference_date = last_date + pd.Timedelta(days=1)
    
    # Calculate Customer Tenure in years from the reference date
    dataset['Customer Tenure'] = (reference_date - dataset['Policy Start Date']).dt.days / 365
    dataset['Claim Frequency'] = dataset['Previous Claims'] / dataset['Insurance Duration']
    # Extract year, month, day, hour, minute, second from 'Policy Start Date'
    dataset['Policy Start Year'] = dataset['Policy Start Date'].dt.year
    dataset['Policy Start Month'] = dataset['Policy Start Date'].dt.month
    dataset['Policy Start Day'] = dataset['Policy Start Date'].dt.day
    dataset['Policy Start Hour'] = dataset['Policy Start Date'].dt.hour
    dataset['Policy Start Minute'] = dataset['Policy Start Date'].dt.minute
    dataset['Policy Start Second'] = dataset['Policy Start Date'].dt.second
    dataset['Null columns'] = dataset.isnull().sum(axis=1)

    del dataset["id"]
    del dataset["Policy Start Date"]

    if "Premium Amount" in dataset.columns:
        dataset["Premium Amount"] = np.log10(dataset["Premium Amount"] + 1)

    dataset["Annual Income log 10"] = np.log10(dataset["Annual Income"])
    dataset["Previous Claims log"] = np.log1p(dataset["Previous Claims"])

    for col in [col for col in dataset.columns if col not in ["Premium Amount","id"]]:
        dataset[f"{col}_null"] = dataset[col].isnull()
    
    for col in categorical_cols:
        if col in dataset.columns:
            dataset[col] =  dataset[col].astype("category")
    
    return dataset

In [186]:
train_data = treat_dataset_pandas_init(full_train_csv_data)
test_data = treat_dataset_pandas_init(full_test_csv_data)

In [187]:
for i in range(0, 10):
    train_data["rand_var_%s" %i] = np.random.rand(len(train_data))
    
    test_data["rand_var_%s" % i] = np.random.rand(len(test_data))

In [188]:
X_train = train_data[[col for col in train_data.columns if col != "Premium Amount"]]
y_train = train_data["Premium Amount"]
X_val = test_data[[col for col in test_data.columns if col != "Premium Amount"]]

In [189]:
from sklearn.metrics import root_mean_squared_log_error

def source_range_root_mean_squared_log_error(y_true, y_pred, *args, **kwargs):
    y_true_orig_range = np.power(10, y_true) - 1
    y_pred_orig_range = np.power(10, y_pred) - 1

    return root_mean_squared_log_error(y_true_orig_range, y_pred_orig_range, *args, **kwargs)

# Lighbm

In [190]:
from sklearn.metrics import root_mean_squared_log_error

In [191]:
def rmsle_objective(y_pred, dataset):
    y_true = dataset.get_label()
    
    y_true = np.power(10, y_true)
    y_pred = np.power(10, y_pred)
    
    # Ensure no negative predictions
    y_pred = np.maximum(y_pred, 1e-15)
    log_diff = np.log(y_pred + 1) - np.log(y_true + 1)
    grad = 2 * log_diff / (y_pred + 1)
    hess = 2 / (y_pred + 1)**2
    
    return grad, hess

def rmsle_metric(y_pred, y_true):    
    y_true = np.power(10, y_true) - 1
    y_pred = np.power(10, y_pred) - 1

    rmsle = root_mean_squared_log_error(y_true, y_pred)
    
    return 'rmsle', rmsle, False  # False because lower is better

In [192]:
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import KFold


In [193]:
y_train.min()

1.3222192947339193

In [194]:
import xgboost as xgb
import lightgbm as lgb


In [195]:
from models import BinnedXGBRegressionModel

In [196]:
X_train.iloc[train_idx]

,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,...,rand_var_0,rand_var_1,rand_var_2,rand_var_3,rand_var_4,rand_var_5,rand_var_6,rand_var_7,rand_var_8,rand_var_9
66961,35.0,Female,1482.0,NaN,1.0,High School,Employed,24.662441,Rural,Basic,...,0.950441,0.749431,0.447514,0.597875,0.657586,0.660331,0.811727,0.219917,0.708700,0.996696
826463,38.0,Female,NaN,Divorced,1.0,PhD,NaN,50.859458,Suburban,Comprehensive,...,0.900767,0.121327,0.647182,0.485303,0.968188,0.357751,0.244816,0.241308,0.338761,0.894308
86341,23.0,Female,3547.0,Divorced,4.0,PhD,NaN,20.847644,Suburban,Comprehensive,...,0.762857,0.155230,0.438627,0.290110,0.239591,0.069354,0.171314,0.818727,0.088609,0.018776
143873,38.0,Female,8073.0,Married,0.0,Master's,Unemployed,28.384782,Suburban,Basic,...,0.679784,0.530853,0.879347,0.530163,0.910157,0.558724,0.146047,0.531897,0.971074,0.766829
513638,56.0,Male,15774.0,Divorced,2.0,Bachelor's,Unemployed,28.973188,Urban,Comprehensive,...,0.101112,0.423065,0.287303,0.825120,0.301917,0.121238,0.683442,0.915017,0.829535,0.558324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1041434,47.0,Male,37156.0,Divorced,0.0,Master's,Self-Employed,12.181688,Rural,Basic,...,0.554456,0.671631,0.955416,0.644458,0.268082,0.512849,0.263681,0.182886,0.121545,0.892401
1131668,52.0,Male,3620.0,Divorced,2.0,PhD,Self-Employed,40.081359,Rural,Premium,...,0.291783,0.566988,0.494830,0.922647,0.073984,0.553243,0.066915,0.636084,0.710856,0.561237
55354,31.0,Male,97447.0,Single,1.0,Master's,NaN,14.844667,Suburban,Basic,...,0.548260,0.491426,0.040335,0.753065,0.439334,0.372999,0.535901,0.577703,0.988914,0.665108
613375,51.0,Female,20125.0,Married,0.0,Bachelor's,Self-Employed,16.141125,Rural,Basic,...,0.291450,0.826849,0.167012,0.695308,0.189377,0.865180,0.059992,0.519954,0.310612,0.620638


In [197]:
from sklearn.model_selection import ShuffleSplit
import numpy as np

# Define a 60-40 split
splitter = ShuffleSplit(n_splits=5, test_size=0.2, random_state=1)

for train_idx, val_idx in splitter.split(X_train):
    print("Starting")
    # Prepare data for LightGBM
    X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]
    
    # Train model
    model = xgb.XGBRegressor(enable_categorical=True, eta=0.05, colsample_bytree=0.75, num_boost_round=500)
    model.fit(X_train_fold, y_train_fold)
    
    train_score = rmsle_metric(model.predict(X_train_fold), y_train_fold)
    val_score = rmsle_metric(model.predict(X_val_fold), y_val_fold)

    print(f"Train: {train_score}, Test: {val_score}")

Starting
Train: ('rmsle', 1.0444940086077172, False), Test: ('rmsle', 1.0444827067325535, False)
Starting
Train: ('rmsle', 1.0441116766274805, False), Test: ('rmsle', 1.04629285034721, False)
Starting
Train: ('rmsle', 1.0442457562790457, False), Test: ('rmsle', 1.0450447657697919, False)
Starting
Train: ('rmsle', 1.0440078597915292, False), Test: ('rmsle', 1.0462594214072105, False)
Starting
Train: ('rmsle', 1.0446441442531227, False), Test: ('rmsle', 1.0437892287695907, False)


In [198]:
importances = pd.Series(model.get_booster().get_score(importance_type='total_gain')).reset_index()
importances.columns = ["column", "gain"]
importances["rand_var"] = importances["column"].str.contains("rand_var")

importances["better_than_probe_avg"] = (importances["gain"] > importances.loc[importances["rand_var"], "gain"].mean()) & ~importances["rand_var"]
importances =importances.sort_values(by="gain")
usable_columns = list(importances[importances["better_than_probe_avg"]]["column"].values)

# LightGBM Limited Features

In [213]:
X_train_fixed = X_train.loc[:, usable_columns]

In [231]:
usable_columns

['Customer Tenure',
 'Marital Status_null',
 'Policy Start Year',
 'Customer Feedback_null',
 'Health Score',
 'Previous Claims',
 'Credit Score',
 'Annual Income']

In [218]:
from sklearn.model_selection import ShuffleSplit
import numpy as np

# Define a 60-40 split
splitter = ShuffleSplit(n_splits=5, test_size=0.2, random_state=1)

for train_idx, val_idx in splitter.split(X_train):
    print("Starting")
    # Prepare data for LightGBM
    X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]
    
    # Train model
    model = xgb.XGBRegressor(enable_categorical=True, eta=0.1
                             , colsample_bytree=1.0, num_boost_round=5000)
    model.fit(X_train_fold, y_train_fold)
    
    train_score = rmsle_metric(model.predict(X_train_fold), y_train_fold)
    val_score = rmsle_metric(model.predict(X_val_fold), y_val_fold)

    print(f"Train: {train_score}, Test: {val_score}")

Starting
Train: ('rmsle', 1.0414319480241936, False), Test: ('rmsle', 1.0443248048531553, False)
Starting
Train: ('rmsle', 1.0409693537040736, False), Test: ('rmsle', 1.0461658721352969, False)
Starting
Train: ('rmsle', 1.04124619189709, False), Test: ('rmsle', 1.0447801409767128, False)
Starting
Train: ('rmsle', 1.0410541920096403, False), Test: ('rmsle', 1.0460724699143016, False)
Starting
Train: ('rmsle', 1.0416228339266616, False), Test: ('rmsle', 1.0436687891507819, False)


In [219]:
importances = pd.Series(model.get_booster().get_score(importance_type='total_gain')).reset_index()
importances.columns = ["column", "gain"]
importances["rand_var"] = importances["column"].str.contains("rand_var")

importances["better_than_probe_avg"] = (importances["gain"] > importances.loc[importances["rand_var"], "gain"].mean()) & ~importances["rand_var"]
importances =importances.sort_values(by="gain")
usable_columns = list(importances[importances["better_than_probe_avg"]]["column"].values)

In [220]:
importances

,column,gain,rand_var,better_than_probe_avg
15,Smoking Status,13.097382,False,False
1,Gender,15.256684,False,False
17,Property Type,38.792526,False,False
9,Policy Type,60.558346,False,False
14,Customer Feedback,72.182533,False,False
3,Marital Status,82.241837,False,False
6,Occupation,84.737862,False,False
5,Education Level,88.832588,False,False
25,Occupation_null,91.203659,False,False
8,Location,95.033760,False,False


In [221]:
from statsmodels.distributions.empirical_distribution import ECDF
import matplotlib.pyplot as plt

def rescale(series, from_min, from_max, to_min, to_max):
    return to_min + ((series - from_min) * (to_max - to_min)) / (from_max - from_min)

In [223]:
model = model.fit(X_train, y_train)

In [225]:
model.save_model("results_201224_1828_limited_columns.pkl")

In [226]:
y_train_pred = model.predict(X_train)#.loc[:, usable_columns])
y_val_pred = model.predict(X_val)#.loc[:, usable_columns])


In [227]:
res_Data = full_test_csv_data[["id"]].copy()

In [228]:
res_Data["pred"] = np.power(10, y_val_pred) - 1

In [229]:
res_Data.columns = ["id","Premium Amount"]

In [230]:
res_Data.to_csv("results_201224_1828_limited_columns.csv",index=False,header=True)